In [2]:
import numpy as np
import pandas as pd

In [3]:
import pickle

In [4]:
import catboost

In [5]:
filename = 'preliminary_trained_model.pkl'

In [6]:
model = pickle.load(open(filename, 'rb'))

In [49]:
valid_df = pd.read_csv('project_data/attempt_4/test.csv')

In [50]:
df_date = pd.read_pickle('macroeconomic.pkl')

In [51]:
df_complete = valid_df.merge(df_date, how='left', left_on='timestamp', right_on='timestamp')

In [58]:
df_complete = df_complete.fillna(0)

In [59]:
submissions = model.predict(df_complete)

In [60]:
submissions

array([ 5924858.05309915,  9916951.06138354,  4892607.41267314, ...,
       -1308363.44179923,  4406463.55423659, 12912744.28472111])

In [61]:
result = pd.DataFrame(valid_df['id'])

In [62]:
result['price_doc_0'] = submissions.tolist()

In [63]:
result = result.set_index('id')

In [64]:
result.head()

,price_doc_0
id,
30474,5.924858e+06
30475,9.916951e+06
30476,4.892607e+06
30477,8.939420e+06
30478,4.757160e+06


In [65]:
result['price_doc'] = result['price_doc_0'].apply(lambda x: round(abs(x), 2) if x < 0 else round(x, 2))

In [66]:
result.drop('price_doc_0', axis = 1, inplace=True)

In [67]:
result.to_csv('submission.csv')

In [68]:
result.shape

(7662, 1)